In [ ]:
!pip install top2vec
!pip install top2vec[sentence_encoders]
!pip install top2vec[sentence_transformers]
!pip install top2vec[indexing]

In [1]:
import logging

logging.basicConfig(format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO, force=True)

In [2]:
import os

import numpy as np
import pandas as pd
from top2vec import Top2Vec
from tqdm import tqdm

2023-07-09 21:12:41.951233: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-09 21:12:41.971687: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-09 21:12:42.122690: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-09 21:12:42.123950: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-09 21:12:43.198367: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
df = pd.read_csv("../parsing/paragraphs.csv.zip", index_col=0)
df.head()

/tmp/ipykernel_12856/668388980.py:1: DtypeWarning: Columns (0,1,2,3,4,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../parsing/paragraphs.csv.zip", index_col=0)


,company,year,sector,report_type,paragraph,original_text,cleaned_text
0,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,0.0,1 ММК,ммк
1,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,1.0,СОЦИАЛЬНЫЙ ОТЧЕТ ММК ЗА 2014 ГОД МАГНИТОГОРСКИ...,социальный отчёт ммк за год магнитогорский мет...
2,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,2.0,www.mmk.ru,NaN
3,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,3.0,СОДЕРЖАНИЕ,содержание
4,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,4.0,Обращение председателя совета директоров.........,обращение председатель совет директор параметр...


In [4]:
df.shape

(6113766, 7)

In [5]:
df = df[df["original_text"].notna()]
df.shape

(5784808, 7)

In [6]:
df.head()

,company,year,sector,report_type,paragraph,original_text,cleaned_text
0,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,0.0,1 ММК,ммк
1,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,1.0,СОЦИАЛЬНЫЙ ОТЧЕТ ММК ЗА 2014 ГОД МАГНИТОГОРСКИ...,социальный отчёт ммк за год магнитогорский мет...
2,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,2.0,www.mmk.ru,NaN
3,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,3.0,СОДЕРЖАНИЕ,содержание
4,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,4.0,Обращение председателя совета директоров.........,обращение председатель совет директор параметр...


In [7]:
hdbscan_args = {"min_cluster_size": 30, "min_samples": 15, "metric": "euclidean"}
umap_args = {"low_memory": True, "random_state": 42}

In [ ]:
%%time
model = Top2Vec(
    documents=df["original_text"].tolist()[: df.shape[0] // 4],
    speed="deep-learn",
    workers=8,
    min_count=0,
    embedding_model="universal-sentence-encoder-multilingual",
    hdbscan_args=hdbscan_args,
    umap_args=umap_args,
)

2023-07-09 21:13:13,496 - top2vec - INFO - Pre-processing documents for training
2023-07-09 21:13:13 - Pre-processing documents for training
/home/samoed/.pyenv/versions/3.10.4/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
2023-07-09 21:13:57,266 - top2vec - INFO - Downloading universal-sentence-encoder-multilingual model
2023-07-09 21:13:57 - Downloading universal-sentence-encoder-multilingual model
2023-07-09 21:13:57 - Using /tmp/tfhub_modules to cache modules.
2023-07-09 21:13:58.603560: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-09 21:13:58.688548: W tensorflow/core/common_runtime

In [ ]:
model.save("model_par.pth")

In [ ]:
topic_words, word_scores, topic_nums = model.get_topics(num_of_topics)
pages_topics = {"topic_id": [], "topic_words": []}
for topic_id, topic_w in tqdm(zip(topic_nums, topic_words)):
    pages_topics["topic_id"].append(topic_id)
    pages_topics["topic_words"].append(", ".join(topic_w))

In [ ]:
pages_topics_df = pd.DataFrame.from_dict(pages_topics)
pages_topics_df.head()

In [ ]:
pages_topics_df.to_excel("pages_topics.xlsx", index=False)

In [ ]:
"test"

In [ ]:
"q"